In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/standings?season=45&conference=12&division=-1&standingstype=division&context=overall&specialteams=false&sortkey=points&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for row in rows[1:]:  # Skip header row
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Team', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PCT', 'PCT2', 'PIM', 'RW', 'ROW', 'STK', 'IN-DIV', 'GPCT', 'P10', 'Nickname']


data_without_first_two = [row[2:] for row in acha_dynamic_stats]

df = pd.DataFrame(data_without_first_two, columns=column_names)


int_columns = ['RW', 'ROW', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PIM']
for column in int_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(int)


float_columns = ['PCT', 'GPCT', 'PCT2']
for column in float_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')


string_columns = ['Team', 'STK', 'IN-DIV', 'P10', 'Nickname']
for column in string_columns:
    df[column] = df[column].astype(str)


df = df[df['Team'] != 'None']
df = df.dropna(subset=['Team'])

print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_wd2_team.csv', index=False)

     Team  GP  W   L  OTL  T  SOW  SOL  PTS  OTW  ...    PCT  PCT2  PIM  RW  \
0     UVA   8  6   2    0  0    0    0   12    0  ...  0.750   0.0   16   6   
1      VT   7  5   2    0  0    0    0   10    0  ...  0.714   0.0   34   5   
2     GWU   6  2   4    0  0    0    0    4    0  ...  0.333   0.0   12   2   
3      GU   7  2   5    0  0    0    0    4    0  ...  0.286   0.0    4   2   
4     WCU  13  2  11    0  0    0    0    4    0  ...  0.154   0.0   49   2   
..    ...  .. ..  ..  ... ..  ...  ...  ...  ...  ...    ...   ...  ...  ..   
75     MC  10  2   7    1  0    0    0    4    0  ...  0.200   0.0   65   2   
76  W2Har  10  0   9    0  1    0    0    1    0  ...  0.050   0.0   27   0   
77   Bent   5  0   5    0  0    0    0    0    0  ...  0.000   0.0   12   0   
78    CHC   6  0   6    0  0    0    0    0    0  ...  0.000   0.0   32   0   
79  W2UNE   0  0   0    0  0    0    0    0    0  ...  0.000   0.0    0   0   

    ROW      STK   IN-DIV   GPCT      P10         N